In [12]:
import pandas as pd
import os

## 회사명 기준으로 파일 나누기
preprocessed_data_로 시작하는 chatgpt 라벨링 데이터를 회사명으로 분류해서 merged_(회사명) 으로 저장

In [3]:
def merge_and_move_files(directory_path):
    file_list = os.listdir(directory_path)

    files_by_company = {}

    # 파일들을 기업 이름에 따라 그룹화
    for file_name in file_list:
        # if file_name[0].isdigit():
        if file_name.startswith("preprocessed_data_"):
            file_path = os.path.join(directory_path, file_name)
            data = pd.read_csv(file_path)
            
            company_name = data['company'].iloc[0]
            
            if company_name not in files_by_company:
                files_by_company[company_name] = []
                
            files_by_company[company_name].append(file_name)

    # 각 기업에 대해 폴더를 만들고 해당 폴더로 각각 파일들을 저장
    for company_name, files in files_by_company.items():
        folder_path = os.path.join(directory_path, "merged")
        os.makedirs(folder_path, exist_ok=True)

        combined_data = pd.concat([pd.read_csv(os.path.join(directory_path, file)) for file in files], axis=0)

        merged_file_path = os.path.join(folder_path, f"merged_{company_name}.csv")
        combined_data.to_csv(merged_file_path, index=False)
        

if __name__ == "__main__":
    target_directory = "/opt/ml/finance_sentiment_corpus"
    merge_and_move_files(target_directory)


## 파일 합치기
merged_(회사명)을 학습 시키기위한 merged_all.csv 만들기

In [ ]:
def merge_csv_files(directory_path, output_file_name):
    file_list = os.listdir(directory_path)
    combined_data = pd.DataFrame()

    for file_name in file_list:
        if file_name.endswith(".csv") and file_name != "merged_all.csv" : # merged_all.csv 제외하고 합치기
            file_path = os.path.join(directory_path, file_name)

            data = pd.read_csv(file_path)
            
            # 혹여나 "labels"로 지정해둔 column 이름 변경
            if "labels" in data.columns and "label" not in data.columns  :
                data["label"] = data["labels"]
                
            data = data[["company", "title", "date", "content_corpus", "label"]]
            combined_data = pd.concat([combined_data, data], axis=0, ignore_index=True)
    
    output_file_path = os.path.join(directory_path, output_file_name)

    # 지우고 다시 만들기
    if os.path.exists(output_file_path):
        os.remove(output_file_path)
        
    combined_data.to_csv(output_file_path, index=False)
    return combined_data

directory_path = "/opt/ml/finance_sentiment_corpus/merged"
output_file_name = "merged_all.csv"
output_file_path = directory_path + output_file_name
    
data = merge_csv_files(directory_path , output_file_name)

## TEST 데이터셋 만들기
26개의 회사에서 20개씩 뽑아 test_dataset 만들기

In [2]:
csv_file_path = "/opt/ml/finance_sentiment_corpus/label_0_to_521.csv"
df = pd.read_csv(csv_file_path)

# 새로운 DataFrame을 저장할 리스트 생성
new_dfs = []

# company 컬럼의 고유한 값들을 추출하여 각 회사별로 20개씩 행을 샘플링하여 새로운 DataFrame으로 생성
for company_name in df['company'].unique():
    company_subset = df[df['company'] == company_name].sample(n=10, random_state=42)  # 20개씩 랜덤 샘플링 (여기서는 random_state를 고정하여 재현성을 위해 사용)
    new_dfs.append(company_subset)

# 새로운 DataFrame을 병합하여 하나의 DataFrame으로 합치기
result_df = pd.concat(new_dfs)

# 새로운 DataFrame을 CSV 파일로 저장
result_csv_file_path = "/opt/ml/finance_sentiment_corpus/26_company_half_labeled.csv"  # 저장할 파일 경로 설정 (적절하게 변경해주세요)
result_df.to_csv(result_csv_file_path, index=False)  # index=False를 지정하여 인덱스를 저장하지 않도록 설정합니다.
